In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [3]:
FEATURES = [
    "gd",
    "ed",
    "gp",
    "isr",
#    "views", 
    "large", 
     "top", 
#    "speed_level",
#    "speed_level 1",
#    "speed_level 2",
#    "speed_level 3",
    "price"]

In [4]:
def get_speed_level(speed_level):
    if speed_level == 1:
        return  [1]
        #return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        return [0.4]
        #return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        return [0.1]
        #return [0,0,1]
        #print X[-1], speed_level
    else:
        return [0]
        #return [0,0,0]
        #print X[-1], speed_level

In [5]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        X+=[[]]
        #gd 0-7
        X[-1]+=[doc['degree'].count(u'Профессор')]
        X[-1]+=[int(doc['top'])]
        X[-1]+=[len(doc['illnesses'])]
        X[-1]+=[len(doc['languages'])]
        X[-1]+=[int(doc['h_index'])]
        X[-1]+=[int(doc['experience'])]
        X[-1]+=[1]
        #ed 7-12
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        X[-1]+=[len(list(set(ids_diad)))]
        X[-1]+=[len(doc['procedures']['diagnostics'])]
        X[-1]+=[len(list(set(ids_oper)))]
        X[-1]+=[len(doc['procedures']['operations'])]
        X[-1]+=[1]
        #gp 12-16
        X[-1]+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
        X[-1]+=[len(all_proc)] #gen_proc_cli
        X[-1]+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
        X[-1]+=[len(clinic['procedures']['operations'])] #gen_oper_cli
        #
        c_id = int(clinic['country_id'])
        if c_id == 1:
            X[-1]+=[1]
        else:
            X[-1]+=[0]
        #X[-1]+=[int(clinic['views'])]
        X[-1]+=[int(clinic['large'])]
        X[-1]+=[int(clinic['top'])]
        #X[-1]+=[get_speed_level(int(clinic['speed_level']))]
        X[-1]+=[round(1-int(clinic_estim['price'])/10.0,4)]
        
    y=[int(clinic_estim['rating'])] * len(all_doc)
    return [X, y]

In [15]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [7]:
def normale(mat):
    for i in range(0,len(mat[0])-1):
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [8]:
def pack(X):
    gd = [0.06, 0.223, 0.372, 0.196, -0.135, 0.426, 8.37]
    ed = [-2.82, -1.104, 0.827, 0.96, 9.471]
    gp = [-3.094, -7.545, 11.025, 9.011]
    new_X = []
    for x in X:
        new_X+= [sum([
            [round(sum(np.array(gd)*np.array(x[0:7])),5)],
            [round(sum(np.array(ed)*np.array(x[7:12])),5)],
            [round(sum(np.array(gp)*np.array(x[12:16])),5)],
            x[16:]
        ],[])]
    return new_X

In [9]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [ ]:
X, y = get_X_y_from(cancer_data)

In [18]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)
X_all=pack(X_all)
normale(X_all)

In [17]:
X_all

[[1, 1, 9, 1, 17, 20, 1, 0, 1, 0, 1, 1, 5, 65, 2, 37, 1, 1, 1, 0.5],
 [1, 1, 3, 0, 0, 0, 1, 0, 0, 0, 1, 1, 5, 65, 2, 37, 1, 1, 1, 0.5],
 [1, 1, 24, 0, 25, 40, 1, 1, 5, 0, 4, 1, 5, 65, 2, 37, 1, 1, 1, 0.5],
 [1, 1, 15, 0, 0, 0, 1, 0, 0, 0, 0, 1, 5, 65, 2, 37, 1, 1, 1, 0.5],
 [1, 0, 4, 0, 18, 20, 1, 1, 4, 0, 6, 1, 5, 65, 2, 37, 1, 1, 1, 0.5],
 [1, 0, 3, 0, 34, 20, 1, 0, 2, 0, 3, 1, 5, 65, 2, 37, 1, 1, 1, 0.5],
 [0, 0, 2, 0, 0, 0, 1, 1, 5, 0, 1, 1, 4, 47, 1, 20, 0, 0, 0, 0.4],
 [1, 0, 2, 0, 14, 0, 1, 1, 3, 0, 4, 1, 5, 48, 1, 17, 0, 1, 0, 0.3],
 [0, 0, 16, 0, 0, 25, 1, 0, 0, 0, 3, 1, 6, 63, 2, 36, 1, 1, 1, 0.7],
 [0, 0, 4, 0, 4, 30, 1, 1, 3, 0, 4, 1, 6, 63, 2, 36, 1, 1, 1, 0.7],
 [0, 0, 1, 0, 49, 0, 1, 1, 4, 0, 0, 1, 3, 16, 0, 4, 0, 0, 0, 0.6],
 [1, 0, 1, 0, 0, 0, 1, 2, 5, 0, 3, 1, 5, 52, 1, 30, 0, 1, 0, 0.3],
 [0, 0, 1, 0, 2, 10, 1, 1, 5, 0, 2, 1, 3, 28, 0, 14, 0, 0, 0, 0.1],
 [0, 0, 16, 0, 0, 25, 1, 0, 0, 0, 3, 1, 6, 63, 2, 36, 1, 1, 1, 0.6],
 [1, 0, 7, 0, 30, 30, 1, 0, 0, 0, 0, 1, 6, 63

In [ ]:
len(X_all[1])

In [ ]:
len(cancer_data[3]['doctors'])

In [ ]:
X_all

In [134]:
for i in range(1,10):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

1
Total: 64, train: 57, test: 7
Residual sum of squares: 10.81
Train absolute: 2.16
Test absolute: 3.07
Absolute to mean: 47.77%
Train variance score: 0.29
Test variance score: -0.66

2
Total: 64, train: 51, test: 13
Residual sum of squares: 8.15
Train absolute: 2.20
Test absolute: 2.57
Absolute to mean: 42.77%
Train variance score: 0.24
Test variance score: 0.07

3
Total: 64, train: 44, test: 20
Residual sum of squares: 7.76
Train absolute: 2.25
Test absolute: 2.47
Absolute to mean: 39.87%
Train variance score: 0.26
Test variance score: -0.04

4
Total: 64, train: 38, test: 26
Residual sum of squares: 6.82
Train absolute: 2.34
Test absolute: 2.41
Absolute to mean: 37.93%
Train variance score: 0.27
Test variance score: -0.02

5
Total: 64, train: 32, test: 32
Residual sum of squares: 7.45
Train absolute: 2.34
Test absolute: 2.29
Absolute to mean: 37.35%
Train variance score: 0.32
Test variance score: -0.09

6
Total: 64, train: 25, test: 39
Residual sum of squares: 7.01
Train absolute: 2.

In [135]:
regr = process_with(X_all, y_all, info=True, ts=0.6)

Total: 64, train: 25, test: 39
Residual sum of squares: 7.01
Train absolute: 2.37
Test absolute: 2.21
Absolute to mean: 34.41%
Train variance score: 0.27
Test variance score: 0.07


In [136]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_all)):    
    print "Real: %f \t Predicted: %f" %(y_all[i], regr.predict(X_all[i]))
    print sum(X_all[i])
    print
    #print y_all[i]
    #print regr.predict(X_all[i])[0]

Real: 10.000000 	 Predicted: 7.704660
5.9451

Real: 10.000000 	 Predicted: 7.070325
5.9051

Real: 10.000000 	 Predicted: 7.309293
5.8955

Real: 10.000000 	 Predicted: 7.252825
5.914

Real: 10.000000 	 Predicted: 6.470389
5.8485

Real: 10.000000 	 Predicted: 6.960433
5.8991

Real: 9.000000 	 Predicted: 5.657148
2.0468

Real: 8.000000 	 Predicted: 5.510347
2.9621

Real: 6.000000 	 Predicted: 7.315954
5.8934

Real: 6.000000 	 Predicted: 6.198178
5.7911

Real: 4.000000 	 Predicted: 2.704850
1.4952

Real: 3.000000 	 Predicted: 5.895186
3.1365

Real: 1.000000 	 Predicted: 3.733958
1.6379

Real: 9.000000 	 Predicted: 7.315954
5.8934

Real: 9.000000 	 Predicted: 6.816188
5.8496

Real: 9.000000 	 Predicted: 6.876036
5.8604

Real: 9.000000 	 Predicted: 6.586185
5.8348

Real: 7.000000 	 Predicted: 5.509737
2.9492

Real: 7.000000 	 Predicted: 5.514804
2.9624

Real: 7.000000 	 Predicted: 4.564887
2.8673

Real: 5.000000 	 Predicted: 7.704660
5.9451

Real: 5.000000 	 Predicted: 7.070325
5.9051

Real:

In [117]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0] #, i[1]

Features sorted by their score:
-9.60471591456
5
10.809400 	 gd
2.475900 	 ed
1.121000 	 gp
0.476800 	 large
5.097000 	 price


In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [61]:
plt.plot(sorted([i[2] for i in X_all]))
plt.show()

In [ ]:
print FEATURES[8]
[i[22] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)